In [1]:
import os
import time
import cv2
import pickle
import subprocess

import numpy as np
import pandas as pd
import torch 
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms as tfs
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader,random_split,TensorDataset

In [2]:

def show_time(func):
    def wrapped(*args,**kwargs):       
        start_time = time.time()
        result = func(*args,**kwargs)
        end_time = time.time()
        
        print("Costs %d seconds\n"%(end_time-start_time))
        
        return result
    return wrapped

class Video(object):
    
    def __init__(self,video_path = None):
        self.video_path = video_path
        self.capture = cv2.VideoCapture(video_path)
        
        self.width = int(self.capture.get(cv2.CAP_PROP_FRAME_WIDTH))
        
        self.height = int(self.capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
        self.shape = (self.width, self.height)
        self.fps = self.capture.get(cv2.CAP_PROP_FPS)
        self.total_frame = self.capture.get(cv2.CAP_PROP_FRAME_COUNT)
        self.duration = int(self.total_frame//self.fps)
        
        
        sec = self.duration % 60
        minu = (self.duration//60)%60
        hr = self.duration // 3600
        self.duration_hms = (hr,minu,sec)
    
    def __len__(self):
        return int(self.total_frame)
    def __repr__(self):
        self.info = 'video path: {0}\nimage size: {1}\nfps:{2}\nframe num: {3}\
            \nduration(s): {4}s {5}\n'.format(self.video_path,\
                    (self.width,self.height),self.fps,self.total_frame,\
                        self.duration,self.duration_hms)
        return self.info

    
    def play(self,start_frame=0,end_frame=None):
        self.capture.set(cv2.CAP_PROP_POS_FRAMES,start_frame)
        
        while(self.capture.isOpened()):
            ret, frame = self.capture.read()
            cv2.imshow('frame',frame)
            
            current_frame_pos = self.capture.get(cv2.CAP_PROP_POS_FRAMES)

            if (cv2.waitKey(1) & 0xFF == ord('q')) or \
                end_frame is not None and current_frame_pos>end_frame:
                break

                
        self.capture.release()
        cv2.destroyAllWindows()
    
    
    def _clip(self,save_path,start_frame=0,end_frame=None,\
              videoWriter=None,new_frame_size = None) -> bool:
        
        # 设置参数
        if end_frame is None or end_frame > self.total_frame:
            end_frame = self.total_frame
            
        assert start_frame < end_frame <= self.total_frame , "起止帧大小不对"
        if videoWriter is None:
            video_fourcc = cv2.VideoWriter_fourcc('M', 'P', '4', '2')
            videoWriter = cv2.VideoWriter(save_path,video_fourcc,\
                                          self.fps,(self.width,self.height))
        
        # 提示视频时长
        print('start:%d\nend:%d\nduration:%ds\n'%(start_frame,end_frame,(end_frame-start_frame)/self.fps))
        
        
        # 开始写视频
        self.capture.set(cv2.CAP_PROP_POS_FRAMES,start_frame)
        while(self.capture.isOpened()):
            current_frame_pos = self.capture.get(cv2.CAP_PROP_POS_FRAMES)

            if current_frame_pos<end_frame:
                ret, frame = self.capture.read()
                
                # 改变frame大小
                if new_frame_size is not None:
                    frame = cv2.resize(frame, new_frame_size,interpolation=cv2.INTER_CUBIC)

                if ret:
                    videoWriter.write(frame)
                else:
                    print('视频解析错误')
                    return False
            else:
                return True
        return False
    
    @show_time
    def clip_by_time(self,save_path,start_time=0,duration=None,end_time=None,\
                     video_writer:cv2.VideoWriter=None,new_frame_size:tuple=None) -> bool:
        '''
        :param save_path: 保存视频路径
        :param start_time: 截取视频开始时间
        :param end_time: 截取视频结束时间
        :param video_writer: <cv2.VideoWriter> 
        :param new_frame_size: 新视频大小

        :return: <bool> 是否成功截取视频
        
        '''
        start_time = self.time2sec(start_time)
        
        if duration is not None:
            duration = self.time2sec(duration)
            end_time = start_time + duration
        else:
            end_time = self.time2sec(end_time)
            
        start_frame = self.time2frame_idx(start_time)
        end_frame = self.time2frame_idx(end_time)
        
        # print("expected time:%ds"%(0.324*duration + 2))
        return self._clip(save_path,start_frame,end_frame,video_writer,new_frame_size)
        
    def get_frame(self,frame_idx):
        if frame_idx > self.total_frame:
            print('frame_idx is greater than total frame')
            return None
        self.capture.set(cv2.CAP_PROP_POS_FRAMES,frame_idx)
        if self.capture.isOpened():
            ret, frame = self.capture.read()
            if ret:
                return frame
        return None
        
    
    def get_frame_by_time(self,time):
        
        return self.get_frame(self.time2frame_idx(time))
    
    @show_time
    def sample_class(self,root_path,time_seg:list,fps,class_name,down_sampling = 1):
        
        # 判断有无文件夹,没有则创建文件夹
        class_path = os.path.join(root_path,class_name)
        if not os.path.exists(class_path):
            os.makedirs(class_path)
            print('Created dir:%s'%class_path)
        
        
        i = 0
        pic_num = len(os.listdir(class_path))
        for start_time,end_time in time_seg:
            start_frame = self.time2frame_idx(start_time)
            end_frame = self.time2frame_idx(end_time)

            for frame_idx in range(start_frame,end_frame,int(fps)):
                idx = i + pic_num
                frame_path = os.path.join(class_path,'%s_.%d.png'%(class_name,idx))
                cv2.imwrite(frame_path,self.get_frame(frame_idx)[::down_sampling,::down_sampling])
                i += 1
        else:
            print('Successfully write all %d images\n%d images in class \'%s\''%(i,idx+1,class_name))
            return True
        
        print('Not all expected pictures are written')
        return False
        
    @staticmethod
    def time2hms(time_by_sec):
        if type(time_by_sec) is tuple:
            return time_by_sec
        sec = time_by_sec % 60
        minu = (time_by_sec//60)%60
        hr = time_by_sec // 3600
        return hr,minu,sec
    
    @staticmethod
    def time2sec(hms):
        if type(hms) is not tuple:
            return hms
        return hms[0]*3600 + hms[1]*60 + hms[2]
    
    @staticmethod
    def sec2hmsstr(sec):
        h = int(sec//3600)
        m = int((sec-3600*h)//60)
        s = float(sec%60)
        return  "%d:%d:%.3f"%(h,m,s)
    
    def frame_idx2time(self,frame_idx,natural = False,to_hms = False):
        
        time = frame_idx/self.fps
        if not to_hms:
            return time.astype(np.int) if natural else time
                
        hr,minu,sec = self.time2hms(time)
        if natural:
            return int(hr),int(minu),int(sec)
        
        return hr,minu,sec

    def time2frame_idx(self,time:float or tuple):
        '''
        时间转换为帧，时间类型可以是float/int/tuple
        '''
        
        
        time = self.time2sec(time)
        frame_idx = int(time * self.fps)
        return frame_idx



    def show_video_info(self):
        print(self)
    def extract_audio(self,out_path,start_time,duration,output_format = 'mp3',hms_format = True):
        if hms_format:
            start_time = ':'.join([str(i) for i in start_time])
            duration = ':'.join([str(i) for i in duration])
        
        command = 'ffmpeg -i "%s" -ss %s -t %s -q:a 0 -map a "%s.%s"'%(self.video_path,start_time,duration,out_path,output_format)
        return subprocess.call(command,shell=True)
        



class CNN(nn.Module):
    
    def __init__(self):
        super(CNN,self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(1,16,kernel_size=3,padding = 1),
            nn.ReLU(),
            
            nn.MaxPool2d(2)  #outputsize = (16,45,80)
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(16,64,kernel_size=3,padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2) #outputsize = (32,22,40)
        )
        self.dense1 = nn.Sequential(
            nn.Linear(64*22*40,32),
            nn.ReLU()
        )
        self.dense2 = nn.Sequential(
            nn.Linear(32,1)
        )
        
        
    def forward(self,x):
        # x = transform(x,2)
        
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0),-1)
        x = self.dense1(x)
        out = self.dense2(x)
        return out

In [3]:

class TennisGame_processor(Video):
    
    def __init__(self,video_path = None,identifier_model_path = None,cls_file=None):
        super(TennisGame_processor,self).__init__(video_path)
        self.model = torch.load(identifier_model_path,map_location=torch.device('cpu'))
        self.cls_file = cls_file
        
    
    def transform(self,image):
        image = image[::2,::2]
        blur = cv2.GaussianBlur(cv2.cvtColor(np.asarray(image),cv2.COLOR_BGR2GRAY),(5,5),0)[::2,::2]
        blur = cv2.GaussianBlur(blur,(3,3),0)[::2,::2]/128
        return blur[np.newaxis,np.newaxis,...].astype(np.float64)
        # return torch.tensor(blur[np.newaxis,...].astype(np.float64), dtype=torch.float32)

    
    
    def get_frames(self,start_frame,f,frame_num,transform = lambda x:x):

        frames = self.transform(self.get_frame(start_frame))
        cur_frame = start_frame
        i = 1
        while i<frame_num:
                        
            cur_frame += f
            if cur_frame >= self.total_frame:
                break
            frames = np.concatenate((frames,self.transform(self.get_frame(cur_frame))))
            i += 1
        return frames

        
    @show_time
    def identify(self,start_time=0,end_time= None,f = 25,batch_size=60,save_path = None) -> np.ndarray:
        
        '''
        输入
        '''
        
        print('identifying ...\n')
        start_frame = int(self.time2frame_idx(start_time))
        end_frame = int(self.time2frame_idx(end_time)) if end_time is not None else int(self.total_frame)
        clss = []
        frame_idxs = []
        frame_idx = start_frame
        while frame_idx< end_frame:
            try:
                frames = self.get_frames(frame_idx,f,batch_size)
            except:
                print('cur_frame_idx%d,end_frame%d'%(frame_idx,end_frame))
                frame_idx += f*batch_size
                continue
            if frames is None:
                break
            frame_tensor = torch.tensor(frames,dtype = torch.float32)
            predict = torch.nn.Sigmoid()(self.model(frame_tensor).squeeze())
            
            
            clss.extend((np.round(predict.detach().numpy()).astype(np.int)))
            frame_idxs.extend(list(range(frame_idx,frame_idx+f*len(predict),f)))
            
            assert len(frame_idxs) == len(clss), 'len(frame_idxs) != len(clss)!\n'
            frame_idx += f*batch_size
        
        
        # np.save()
        frame_idxs = np.array(frame_idxs)
        times = self.frame_idx2time(np.array(frame_idxs),True)[np.newaxis,...]
        result = np.concatenate((times, np.array([frame_idxs,clss],dtype = np.int)))
        if save_path is not None:
            np.save(save_path,result)
        return result

    

    def play_match(self,cls_file = None,add_cls_info = True,show = True, write = True):
        if cls_file is not None:
            self.cls_file = cls_file
        elif self.cls_file is None:
            print("No cls file found\n")
            return None
        cls_info = np.load(self.cls_file)
        start_frame = cls_info[1][0]
        end_frame = cls_info[1][-1]
        
        if write:
            video_fourcc = cv2.VideoWriter_fourcc('M', 'P', '4', '2')
            videoWriter = cv2.VideoWriter('./zzq.avi',video_fourcc,\
                                          self.fps,(self.width,self.height)) 
            
        self.capture.set(cv2.CAP_PROP_POS_FRAMES,start_frame)
        i = j = 0
        interv = cls_info[1][1:]-cls_info[1][:-1]
        while(self.capture.isOpened()):
            ret, frame = self.capture.read()
            
            if add_cls_info:
                if not i%interv[j] and j< len(interv)-1:
                    j+=1
                c = cls_info[2][j]
                show_info = 'Tennis Court' if c else 'Background'
                if c:
                    cv2.putText(frame, show_info, (100,100), cv2.FONT_HERSHEY_COMPLEX,1.2, (0, 255, 0))
                else:
                    cv2.putText(frame, show_info, (100,100), cv2.FONT_HERSHEY_COMPLEX,1.2, (255,0 , 255))
            
            if show:
                cv2.imshow(self.video_path.split('\\/')[-1],frame)
            if write:
                videoWriter.write(frame)

            current_frame_pos = self.capture.get(cv2.CAP_PROP_POS_FRAMES)

            if (cv2.waitKey(1) & 0xFF == ord('q')) or \
                end_frame is not None and current_frame_pos>end_frame:
                break
            i+=1
        if show:
            self.capture.release()
            cv2.destroyAllWindows()
    
        
    def get_rounds(self,cls_file = None,drop_short = True):
        '''
        

        Parameters
        ----------
        cls_file : str, optional
            存储frame_idx,time,class的文件路径. The default is None.
        drop_short : bool, optional
            是否去掉短片段. The default is True.

        Returns
        -------
        rounds_time : np.ndarray
            所有回合的起始终止时间对.
        rounds_frame : TYPE
            所有回合的起始终止帧对.

        '''
        if self.cls_file is None:
            self.cls_file = cls_file 
        cls_info = np.load(cls_file)
        status_trans = np.zeros_like(cls_info[2])
        status_trans[1:] = cls_info[2][1:]-cls_info[2][:-1]
        start_time = np.argwhere(status_trans==1)
        end_time = np.argwhere(status_trans==-1)

        if len(start_time) and len(end_time):
            if end_time[0][0] < start_time[0][0]:
                start_time = np.concatenate((np.array([[0]]), start_time))
            if end_time[-1][0] < start_time[-1][0]:
                end_time = np.concatenate((end_time,np.array([[len(status_trans)]]) ))
        
        
        assert (start_time < end_time).all(), "start time must be less than end time"
        rounds_idx = np.concatenate((start_time, end_time-1),axis = 1)
        
        rounds_frame = cls_info[1][rounds_idx]
        rounds_time = cls_info[0][rounds_idx]
        if drop_short:
            rounds_frame = rounds_frame[np.argwhere((rounds_time[:,1]-rounds_time[:,0])!=0).squeeze()]
            rounds_time = rounds_time[np.argwhere((rounds_time[:,1]-rounds_time[:,0])!=0).squeeze()]
            
            rounds_frame = rounds_frame[np.argwhere((rounds_time[:,1]-rounds_time[:,0])!=1).squeeze()]
            
            rounds_time = rounds_time[np.argwhere((rounds_time[:,1]-rounds_time[:,0])!=1).squeeze()]
        return rounds_time,rounds_frame
        
    
    @show_time
    def clip_rounds(self,save_root):
        if not os.path.exists(save_root):
            os.makedirs(save_root)
        self.rounds_root = save_root
        
        if self.cls_file is None:
            self.cls_file = r'../rounds/rounds.npy'

            self.identify(save_path = self.cls_file)
        
        rounds_time,rounds_frame = self.get_rounds(self.cls_file)
        for i,(start_frame,end_frame) in enumerate(rounds_frame):
            save_path = os.path.join(self.rounds_root,'round%d_%d_%d.avi'%(i,start_frame,end_frame))
            if self._clip(save_path,start_frame,end_frame):
                print('saved',save_path)

        print('%d rounds are saved\n'%(i+1))
        return 
    
    
    def get_file_and_clip_fucking_video_to_part(self):
        pass
    def random_frame(self,show=False):
        frame_idx = np.random.randint(self.total_frame)
        img = self.get_frame(frame_idx)
        if show:
            plt.imshow(img)
        return img

    
    
    
    
    # 输入视频，将视频的分段剪出来，得到场景分类信息，删掉低于2秒的片段：

    def rounds_cls_info(self,start_time=0,end_time= None,f = 25,batch_size=60,save_path = None,drop_short_thre = 2.):
        
#         cls_info = video.identify(start_time,end_time,f,batch_size,save_path)
        # 删掉
        path = '../scene_cls_info.npy'

        rounds_time,rounds_frame = video.get_rounds(path)
        
        
        df = pd.DataFrame(columns=['game','round','start time', 'end time','start sec','end sec','duration','interval','start frame','end frame','last time','feature','label'])
        print(rounds_time)
        te = 0
        for idx,(s,e) in enumerate(rounds_time):
            
            r = dict()
            r['game'] = self.video_path
            r['round'] = idx
            r['start time'] = self.sec2hmsstr(s)
            r['end time'] = self.sec2hmsstr(e)
            r['start sec'] = s
            r['end sec'] = e
            r['start frame'],r['end frame'] = rounds_frame[idx]
            
            r['duration'] = e-s
            
            df = df.append(r,ignore_index=True)
            
            
            
        df = df[df['end sec'] - df['start sec']>=drop_short_thre]        
        self.r_cls_df = df
        
        return df
        
    # 建文件夹，提取相应df的音频
    def save_audio(self,path,thred_duration = (1,3)):
        '''
        输入路径，输出音频特征
        Parameters
        path:要存放round音频的路径
        
        '''
        if not os.path.exists(path):
            os.mkdir(path)
        min_dur,max_dur = thred_duration
        
        
        
        for index, row in self.r_cls_df.iterrows():
            start = row['end time']
            
            interv = max_dur
            if index+1< len(self.r_cls_df):
                interv = self.r_cls_df['start sec'][index+1] - self.r_cls_df['end sec'][index]
                row['interval'] = interv
            clip_duration = np.clip(interv,min_dur,max_dur)
            clip_duration = "0:0:%d"%clip_duration
            
            rounds_file = "round%d"%(index)
            
            output_file = os.path.join(path,rounds_file)
            self.extract_audio(output_file,start,clip_duration,output_format = 'wav',hms_format=False)
            
            print('%s saved, %s'%(output_file,clip_duration))
            
    
    
    # 提取特征
    def FeatureExtraction(self,config,audio,output):
        
        
        if config == 'audiovisual':
            config_path = "/home/opensmile-3.0-linux-x64/config/audiovisual/audio.conf"
            
        elif config == 'mfcc':
            config_path = "/home/opensmile-3.0-linux-x64/config/mfcc/MFCC12_0_D_A_Z.conf"
        elif config == 'emotion':
            config_path = "/home/opensmile-3.0-linux-x64/config/is09-13/IS10_paraling.conf"
        elif config == 'IS09':
            config_path = "/home/opensmile-3.0-linux-x64/config/is09-13/IS09_emotion.conf"
            
        else:
            config_path = config
        os.system("SMILExtract -C %s -I \"%s\" -O \"%s\""%(config_path,audio,output))
    
    def process_features(self,source,des_format = 'npy',save=True,save_path = None):
        
        
        '''
        通过txt文件，将特征存储为npy文件
        '''
        
        origin_format = source.split('.')[-1]
        
        if origin_format == 'txt':
            with open(source,'r') as f:

                last_line = f.readlines()[-1]
            features=np.array([eval(i) for i in last_line.split(',')[1:-1]])


        elif origin_format=='csv':
            pass
        
        if des_format == 'npy' and save:
            np.save(save_path,features)
        return features
    
    
    def extract_features_in_batch(self,path):
        '''
        将所有回合的音频特征存储为npy文件
        '''
        round_path_list = list(os.walk(path))[0][2]
        
        for r_audio in round_path_list:
            if '.wav' not in r_audio:
                continue
            idx = r_audio.replace('round','').replace('.wav','')
            print(idx)
            input_path = os.path.join(path,r_audio)
            output_path = os.path.join(path,r_audio.replace('.wav','.txt'))
            self.FeatureExtraction('IS09',input_path,output_path)
            self.process_features(output_path,save_path = output_path.replace('.txt','.npy'))
        
        
            print("finished:",idx)
            
    def compile_data(self,path):
        
        round_path_list = list(os.walk(path))[0][2]
        
        for r_audio in round_path_list:
            if '.npy' not in r_audio:
                continue
            idx = int(r_audio.replace('round','').replace('.npy',''))
            self.r_cls_df['feature'][idx] = np.load(os.path.join(path,r_audio))
            print("finished:",idx)
        

        return df
    
    # 输入神经网络里，进行打分
    def ScoreRound(self,model):
        fea_dim = 384
        
        # 删掉
        model =  nn.Sequential(
            nn.Linear(fea_dim,32),
            nn.ReLU(),
            nn.Linear(32,64),
            nn.ReLU(),
            nn.Linear(64,1),
        )
        
        for index, row in self.r_cls_df.iterrows():
            if row['feature'] is not np.nan:
                row['label'] = nn.Sigmoid()(model(torch.tensor(row['feature'],dtype=torch.float))).detach().numpy()[0]
                

    
    # 将对应高分的集锦剪辑成为集锦
    
    def ClipHighlight(self,save_path):
        rank = np.argsort(self.r_cls_df['label'].values)
        num = len(self.r_cls_df)
        best = rank[-num:]
        
        print(best)
        # 写视频
        video_fourcc = cv2.VideoWriter_fourcc('M', 'P', '4', '2')
        videoWriter = cv2.VideoWriter(save_path,video_fourcc,\
                                      self.fps,(self.width,self.height))
        for i in best:
            
            start_frame = self.r_cls_df['start frame'][i]
            end_frame= self.r_cls_df['end frame'][i]
            print(i,start_frame,end_frame)
            # 开始写视频
            self.capture.set(cv2.CAP_PROP_POS_FRAMES,start_frame)
            while(self.capture.isOpened()):
                current_frame_pos = self.capture.get(cv2.CAP_PROP_POS_FRAMES)
                print(current_frame_pos)
                if current_frame_pos<end_frame:
                    ret, frame = self.capture.read()

                    # 改变frame大小
                    new_frame_size = None
                    if new_frame_size is not None:
                        frame = cv2.resize(frame, new_frame_size,interpolation=cv2.INTER_CUBIC)

                    if ret:
                        videoWriter.write(frame)
                    else:
                        print('视频解析错误')
                        break

                else:
                    break
        return 
    
    
    
    
    

# 输入视频，将视频的分段剪出来，得到场景分类信息，删掉低于2秒的片段：

In [9]:
video_path = '/home/expend_toshiba/xieyuning/video/2018查尔斯顿赛第一轮布沙尔VS埃拉尼 回放.ts'
# video_path = r'../video/2019WTA印第安维尔斯站女单决赛科贝尔VS安德莱斯库 英文录播.ts'
video = TennisGame_processor(video_path,'../model/best_model.pkl')
path = '../scene_cls_info.npy'
# cls_info = video.identify(start_time=(0,15,0),end_time= (0,16,0),f = 25,batch_size=60,save_path = path)
df = video.rounds_cls_info()
video.save_audio('../testing')
video.extract_features_in_batch('../testing')
video.compile_data('../testing')
video.ScoreRound(model=0)
video.ClipHighlight('../r2.avi')

[[900 906]
 [912 914]
 [924 930]
 [938 950]]
../testing/round0 saved, 0:0:3
../testing/round1 saved, 0:0:3
../testing/round2 saved, 0:0:3
../testing/round3 saved, 0:0:3
0
finished: 0
3
finished: 3
2
finished: 2
1
finished: 1
finished: 3
finished: 1
finished: 2
finished: 0
[3 2 0 1]
3 23450 23750
23450.0
23451.0
23452.0
23453.0
23454.0
23455.0
23456.0
23457.0
23458.0
23459.0
23460.0
23461.0
23462.0
23463.0
23464.0
23465.0
23466.0
23467.0
23468.0
23469.0
23470.0
23471.0
23472.0
23473.0
23474.0
23475.0
23476.0
23477.0
23478.0
23479.0
23480.0
23481.0
23482.0
23483.0
23484.0
23485.0
23486.0
23487.0
23488.0
23489.0
23490.0
23491.0
23492.0
23493.0
23494.0
23495.0
23496.0
23497.0
23498.0
23499.0
23500.0
23501.0
23502.0
23503.0
23504.0
23505.0
23506.0
23507.0
23508.0
23509.0
23510.0
23511.0
23512.0
23513.0
23514.0
23515.0
23516.0
23517.0
23518.0
23519.0
23520.0
23521.0
23522.0
23523.0
23524.0
23525.0
23526.0
23527.0
23528.0
23529.0
23530.0
23531.0
23532.0
23533.0
23534.0
23535.0
23536.0
23537.0

In [ ]:
video.r_cls_df


[[900 906]
 [912 914]
 [924 930]
 [938 950]]


In [78]:
df

,game,round,start time,end time,start sec,end sec,duration,interval,start frame,end frame,last time,feature,label
0,/home/expend_toshiba/xieyuning/video/2018查尔斯顿赛...,0,0:15:0.000,0:15:6.000,900,906,6,6,22500,22650,NaN,NaN,NaN
1,/home/expend_toshiba/xieyuning/video/2018查尔斯顿赛...,1,0:15:12.000,0:15:14.000,912,914,2,10,22800,22850,NaN,NaN,NaN
2,/home/expend_toshiba/xieyuning/video/2018查尔斯顿赛...,2,0:15:24.000,0:15:30.000,924,930,6,8,23100,23250,NaN,NaN,NaN
3,/home/expend_toshiba/xieyuning/video/2018查尔斯顿赛...,3,0:15:38.000,0:15:50.000,938,950,12,NaN,23450,23750,NaN,NaN,NaN


# 为了中期报告，把识别成功的片段剪出来

In [5]:
video_path = '/home/expend_toshiba/xieyuning/video/2018查尔斯顿赛第一轮布沙尔VS埃拉尼 回放.ts'
# video_path = r'../video/2019WTA印第安维尔斯站女单决赛科贝尔VS安德莱斯库 英文录播.ts'
video = TennisGame_processor(video_path,'../model/best_model.pkl')
path = '../video/info.npy'
video.identify(start_time=(0,15,0),end_time= (0,16,0),f = 25,batch_size=60,save_path = path)
video.play_match(path,True,False,True)


identifying ...

Costs 34 seconds



In [48]:
a = 100
np.clip(a,1,10)

10

In [84]:
'hello' in 'helloworld'

True

In [22]:
video.r_cls_df.sort_values(['label'],ascending= False).iloc[:,1:]

,round,start time,end time,start sec,end sec,duration,interval,start frame,end frame,last time,feature,label
1,1,0:15:12.000,0:15:14.000,912,914,2,10,22800,22850,NaN,"[0.0201733, 0.001315607, 0.0188577, 104.0, 84....",0.995534
0,0,0:15:0.000,0:15:6.000,900,906,6,6,22500,22650,NaN,"[0.0442924, 0.00195311, 0.04233929, 149.0, 195...",0.986446
2,2,0:15:24.000,0:15:30.000,924,930,6,8,23100,23250,NaN,"[0.02758174, 0.002372314, 0.02520943, 206.0, 6...",0.812747
3,3,0:15:38.000,0:15:50.000,938,950,12,NaN,23450,23750,NaN,"[0.04239401, 0.002544475, 0.03984954, 101.0, 2...",0.626455
